# Подготовка данных для нейронной сети features SPAN

Используется соотношение открытого интереса по Call и Put опционам.  
День недели.  
Target по направлению 1('up'), 0('down').  
Запись результатов в файл `nn_features_and_target.csv`  

In [302]:
from pathlib import Path
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import sys
import importlib

sys.dont_write_bytecode = True

In [303]:
db_path: Path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day_2014.db')

# Подключение к базе данных
conn = sqlite3.connect(db_path)

# Чтение данных из БД в DataFrame
df_fut = pd.read_sql_query("SELECT `TRADEDATE`, `OPEN`, `LOW`, `HIGH`, `CLOSE` FROM Futures", conn)
df_opt = pd.read_sql_query("SELECT `TRADEDATE`, `OPENPOSITION`, `OPTIONTYPE`, `STRIKE`, `LSTTRADE` FROM Options", conn)

# Закрытие соединения
conn.close()

df_fut['DIRECTION'] = (df_fut['CLOSE'] > df_fut['OPEN']).astype(int)

df_fut

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,DIRECTION
0,2014-01-06,144450.0,138410.0,144990.0,138530.0,0
1,2014-01-08,138610.0,138020.0,139860.0,138860.0,1
2,2014-01-09,138750.0,138300.0,139670.0,138680.0,0
3,2014-01-10,138690.0,137210.0,140130.0,140110.0,1
4,2014-01-13,140090.0,139600.0,141140.0,140270.0,1
...,...,...,...,...,...,...
2806,2025-03-10,112410.0,112200.0,116200.0,116070.0,1
2807,2025-03-11,116050.0,114650.0,118090.0,116510.0,1
2808,2025-03-12,116720.0,114000.0,117850.0,115170.0,0
2809,2025-03-13,115190.0,112610.0,116840.0,116560.0,1


In [304]:
df_opt

,TRADEDATE,OPENPOSITION,OPTIONTYPE,STRIKE,LSTTRADE
0,2014-01-06,0,C,100000,2014-01-15
1,2014-01-06,0,C,100000,2014-02-17
2,2014-01-06,2,C,100000,2014-03-17
3,2014-01-06,388,P,100000,2014-01-15
4,2014-01-06,500,P,100000,2014-02-17
...,...,...,...,...,...
854165,2025-03-14,3320,P,92500,2025-03-20
854166,2025-03-14,1006,C,95000,2025-03-20
854167,2025-03-14,1820,P,95000,2025-03-20
854168,2025-03-14,980,C,97500,2025-03-20


In [305]:
df_fut[['TRADEDATE']] = df_fut[['TRADEDATE']].apply(pd.to_datetime)
df_opt[['TRADEDATE', 'LSTTRADE']] = df_opt[['TRADEDATE', 'LSTTRADE']].apply(pd.to_datetime)

df_opt = df_opt[df_opt.TRADEDATE < df_opt.LSTTRADE]

df_opt

,TRADEDATE,OPENPOSITION,OPTIONTYPE,STRIKE,LSTTRADE
0,2014-01-06,0,C,100000,2014-01-15
1,2014-01-06,0,C,100000,2014-02-17
2,2014-01-06,2,C,100000,2014-03-17
3,2014-01-06,388,P,100000,2014-01-15
4,2014-01-06,500,P,100000,2014-02-17
...,...,...,...,...,...
854165,2025-03-14,3320,P,92500,2025-03-20
854166,2025-03-14,1006,C,95000,2025-03-20
854167,2025-03-14,1820,P,95000,2025-03-20
854168,2025-03-14,980,C,97500,2025-03-20


In [306]:
# Мой код
# step_strike = 2500

# df_rez = pd.DataFrame()

# for row in df_fut.itertuples():  # Перебирает построчно DF с фьючерсами
#     # Выбор из DF с опционами только опционов на дату фьючерса
#     df = df_opt[df_opt.TRADEDATE == row.TRADEDATE]
#     # print(df.to_string(max_rows=4, max_cols=10))
    
#     # DF с опционами put
#     df_p = (
#         df.query('OPTIONTYPE ==  "P"')
#         .groupby(['STRIKE'], as_index=False)
#         .agg({'OPENPOSITION': 'sum'})
#         .sort_values(['STRIKE'], ascending=True)
#         .rename(columns={'OPENPOSITION': 'oi_p'})
#     )
#     # print(df_p.to_string(max_rows=4, max_cols=10))
#     # print(df_p.shape)

#     if len(df_p) == 0:
#         continue
    
#     # DF с опционами call
#     df_c = (
#         df.query('OPTIONTYPE ==  "C"')
#         .groupby(['STRIKE'], as_index=False)
#         .agg({'OPENPOSITION': 'sum'})
#         .sort_values(['STRIKE'], ascending=True)
#         .rename(columns={'OPENPOSITION': 'oi_c'})
#     )
#     # print(df_c.to_string(max_rows=4, max_cols=10))
#     # print(df_c.shape)
    
#     # Временный DF со страйками (для последующего слияния), чтобы исключить пропуски страйков
#     df_tmp = pd.DataFrame(columns=['STRIKE'])
#     for st in range(df.STRIKE.min(), df.STRIKE.max() + step_strike, step_strike):
#     # for st in np.arange(df.STRIKE.min(), df.STRIKE.max() + step_strike, step_strike):
#     # for st in np.linspace(df.STRIKE.min(), df.STRIKE.max() + step_strike, step_strike):
#         # Новое значение для добавления
#         new_row = pd.DataFrame({'STRIKE': [st]})
#         # Добавление новой строки в DataFrame с помощью concat
#         df_tmp = pd.concat([df_tmp, new_row], ignore_index=True)
#     # print(df_tmp.to_string(max_rows=4, max_cols=10))
    
#     # Выполнение полного внешнего соединения DF'ов с опционами call и put по столбцу STRIKE
#     merged_df = pd.merge(df_p, df_c, on='STRIKE', how='outer')
#     merged_df = pd.merge(merged_df, df_tmp, on='STRIKE', how='outer')
#     # Приведение типов с помощью infer_objects
#     merged_df = merged_df.infer_objects(copy=False)
    
#     merged_df = merged_df.fillna(0)  # Пустые значения заполняем "0"
#     # Приведение типов к int
#     merged_df[['STRIKE', 'oi_c', 'oi_p']] = merged_df[['STRIKE', 'oi_c', 'oi_p']].astype(int)
#     # print(merged_df.to_string(max_rows=4, max_cols=10))
#     # print(merged_df.shape)
    
#     # Переделываем колонку с ОИ по call в колонку с накопленной суммой ОИ
#     merged_df['oi_c'] = merged_df['oi_c'].cumsum()
#     # Переделываем колонку с ОИ по put в колонку с накопленной суммой ОИ в обратном порядке
#     merged_df['oi_p'] = merged_df.iloc[::-1]['oi_p'].cumsum()[::-1]
#     # print(merged_df.to_string(max_rows=4, max_cols=10))
    
#     # Записываем в переменную значение цены CLOSE фьючерса
#     price_close = df_fut.loc[df_fut['TRADEDATE'] == row.TRADEDATE, 'CLOSE'].values[0]
#     # Расчитываем ближайший страйк к цене CLOSE
#     nearest_strike = round(price_close / step_strike) * step_strike
#     # print(price_close, nearest_strike)
    
#     # Список индексов со значением nearest_strike из поля merged_df['STRIKE']
#     index_lst = merged_df.index[merged_df['STRIKE'] == nearest_strike].tolist()
#     # Известный индекс строки (берем 0)
#     index_nearest = index_lst[0]
#     # Определение диапазона строк
#     start_index = max(0, index_nearest - 10)
#     end_index = min(len(df), index_nearest + 10 + 1)
#     # Получение 10 строк до и 10 строк после строки с известным индексом ближайшего страйка
#     subset_df = merged_df.iloc[start_index:end_index]
#     subset_df = subset_df.copy()
#     # Создание колонки соотношений (разницы) отрытого интереса Call и Put
#     subset_df['oi'] = subset_df.apply(lambda x: x.oi_p - x.oi_c if x.STRIKE < price_close else x.oi_c - x.oi_p, axis=1)
#     # print(subset_df.to_string(max_rows=40, max_cols=10))

#     # Извлекаем колонку 'oi' из subset_df
#     column_oi_arr = subset_df['oi'].values
#     # Нормализация значений в диапазон [0, 1]
#     arr_min = np.min(column_oi_arr)
#     arr_max = np.max(column_oi_arr)
#     normalized_arr = (column_oi_arr - arr_min) / (arr_max - arr_min)

#     column_as_row = np.array(
#         [row.TRADEDATE, row.OPEN, row.LOW, row.HIGH, row.CLOSE, nearest_strike] + list(normalized_arr)
#         )
#     column_as_row = np.append(column_as_row, row.DIRECTION)
    
#     # Добавляем колонку как строку в DataFrame df_rez
#     # Для этого нам нужно транспонировать DataFrame, добавить строку и снова транспонировать обратно
#     df_rez = df_rez.T
#     df_rez[len(df_rez.columns)] = column_as_row
#     df_rez = df_rez.T
#     # print(df_rez.to_string(max_rows=40, max_cols=25))

#     # break

# # Преобразование колонки в тип datetime
# df_rez[0] = pd.to_datetime(df_rez[0])
# df_rez[0] = df_rez[0].dt.date  # Оставляем только дату
# df_rez = df_rez.rename(
#     columns={
#         0: 'TRADEDATE', 1:'OPEN', 2:'LOW', 3:'HIGH', 4:'CLOSE', 5:'ZERO_STRIKE', 6:-25000, 
#         7:-22500, 8:-20000, 9:-17500, 10:-15000, 11:-12500, 12:-10000, 13:-7500, 14:-5000, 
#         15:-2500, 16:0, 17:2500, 18:5000, 19:7500, 20:10000, 21:12500, 22:15000, 23:17500, 
#         24:20000, 25:22500, 26:25000, 27:'DIRECTION'
#         }
#     )

# print(df_rez.to_string(max_rows=8, max_cols=30))
# # print(df_rez.shape)

In [307]:
# Код оптимизированный ChatGPT
step_strike = 2500
df_rez = pd.DataFrame()

for row in df_fut.itertuples(index=False):  
    df = df_opt[df_opt.TRADEDATE == row.TRADEDATE]

    df_p = (df[df.OPTIONTYPE == "P"]
            .groupby('STRIKE', as_index=False)['OPENPOSITION']
            .sum()
            .rename(columns={'OPENPOSITION': 'oi_p'}))

    df_c = (df[df.OPTIONTYPE == "C"]
            .groupby('STRIKE', as_index=False)['OPENPOSITION']
            .sum()
            .rename(columns={'OPENPOSITION': 'oi_c'}))

    if df_p.empty and df_c.empty:
        continue

    # Создание DataFrame со страйками
    df_tmp = pd.DataFrame({'STRIKE': np.arange(df.STRIKE.min(), df.STRIKE.max() + step_strike, step_strike)})

    # Объединение данных
    merged_df = df_tmp.merge(df_p, on='STRIKE', how='left').merge(df_c, on='STRIKE', how='left').fillna(0)

    # Приведение типов
    merged_df[['oi_c', 'oi_p']] = merged_df[['oi_c', 'oi_p']].astype(np.int64)

    # Накопленные суммы
    merged_df['oi_c'] = merged_df['oi_c'].cumsum()
    merged_df['oi_p'] = merged_df['oi_p'][::-1].cumsum()[::-1]

    price_close = df_fut.loc[df_fut['TRADEDATE'] == row.TRADEDATE, 'CLOSE'].values[0]
    nearest_strike = round(price_close / step_strike) * step_strike

    if nearest_strike not in merged_df['STRIKE'].values:
        continue  # Если страйк отсутствует, пропускаем

    index_nearest = merged_df.index[merged_df['STRIKE'] == nearest_strike][0]
    start_index = max(0, index_nearest - 10)
    end_index = min(len(merged_df), index_nearest + 11)

    subset_df = merged_df.iloc[start_index:end_index].copy()
    subset_df['oi'] = np.where(subset_df['STRIKE'] < price_close,
                               subset_df['oi_p'] - subset_df['oi_c'],
                               subset_df['oi_c'] - subset_df['oi_p'])

    column_oi_arr = subset_df['oi'].values
    arr_min, arr_max = column_oi_arr.min(), column_oi_arr.max()
    normalized_arr = (column_oi_arr - arr_min) / (arr_max - arr_min) if arr_max != arr_min else np.zeros_like(column_oi_arr)

    row_data = [row.TRADEDATE, row.OPEN, row.LOW, row.HIGH, row.CLOSE, nearest_strike] + list(normalized_arr) + [row.DIRECTION]
    
    df_rez = pd.concat([df_rez, pd.DataFrame([row_data])], ignore_index=True)

# Преобразование даты
df_rez[0] = pd.to_datetime(df_rez[0]).dt.date

# Переименование колонок
df_rez = df_rez.rename(columns={
    0: 'TRADEDATE', 1: 'OPEN', 2: 'LOW', 3: 'HIGH', 4: 'CLOSE', 5: 'ZERO_STRIKE',
    **{i + 6: v for i, v in enumerate(range(-25000, 27500, 2500))}, len(df_rez.columns) - 1: 'DIRECTION'
})

columns = [i for i in range(-25000, 25001, 2500)]  # Генерация имен столбцов
# Сдвиг значений в заданных колонках
shifted_columns = columns  # Указываем столбцы, которые хотим сдвигать
for col in shifted_columns:
    df_rez[col] = df_rez[col].shift(1)  # Сдвигаем значения на 1 строку вниз

print(df_rez.to_string(max_rows=8, max_cols=30))


       TRADEDATE      OPEN       LOW      HIGH     CLOSE  ZERO_STRIKE    -25000    -22500    -20000    -17500    -15000    -12500    -10000     -7500     -5000     -2500         0      2500      5000      7500     10000     12500     15000     17500     20000     22500     25000  DIRECTION
0     2014-01-06  144450.0  138410.0  144990.0  138530.0       137500       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN          0
1     2014-01-08  138610.0  138020.0  139860.0  138860.0       140000  1.000000  0.999997  0.992860  0.992849  0.963177  0.962245  0.912755  0.907981  0.456148  0.441896  0.272335  0.000000  0.125420  0.176985  0.216272  0.276157  0.293606  0.368088  0.375285  0.402358  0.402528          1
2     2014-01-09  138750.0  138300.0  139670.0  138680.0       137500  1.000000  0.992872  0.992861  0.962980  0.962041  0.9047

In [308]:
# Удаляем строки с NaN и сбрасываем индексы
df_rez.dropna(inplace=True)  # Удаляем строки с NaN
df_rez.reset_index(inplace=True, drop=True)  # Сбрасываем индексы

# Сохранение DF в файл без индекса
df_rez.to_csv(fr'span_nn_prepare.csv', index=False)